In [4]:
import pandas as pd

#upload the file
predicted_value = pd.read_excel("clean data before python.xlsx",sheet_name="predicted_value",header=None, skiprows=11)
metric = pd.read_excel("clean data before python.xlsx",sheet_name="predicted_value",header=None, nrows=10).T
auditInfo = pd.read_excel("clean data before python.xlsx",sheet_name="Audit Information", skiprows=3)
metric.columns = metric.iloc[0]
metric = metric.drop(0)

In [17]:
#transform from pivot to raw data
dfAll = pd.DataFrame()
for i in range(predicted_value.shape[0]) :
    tempSeries = predicted_value.iloc[i]
    theID = tempSeries[0]
    dfTemp = metric
    dfTemp["New_Audit_id"] = theID
    dfTemp["predictedValue"] = tempSeries[1:]
    dfAll = pd.concat([dfAll,dfTemp])
#make new ID
dfAll["predicted_id"] = [ a+ "_" + str(b) for a,b in zip(dfAll["New_Audit_id"],dfAll["Remarks"])]
#rearrange data
dfAll = dfAll[["predicted_id",'New_Audit_id', 'Metric:', 'Case:', 'Depth:', 'Energy:',
      'Energy_FFF :', 'Gantry:', 'Mode:', 'Unidentified_column :',
      'predictedValue']] 
dfAll = dfAll.drop_duplicates()
dfFilteres = dfAll[dfAll["predictedValue"].isna() == False]
combinedAll = auditInfo.merge(dfFilteres,on="New_Audit_id")

In [20]:
combinedAll = combinedAll[['predicted_id','New_Audit_id', 'Audit_ID', 'Facility_ID',
       'Status', 'Auditor1', 'Auditor2', 'Auditor3', 'Au date', 'Manuf',
       'Model', 'TPS', 'Algorithm', 'Matched', 'k_Q Fac.', 'k_Q ACDS', 'Array',
       'S.N.', 'audit', '6_k_user', '10_k_user', '15_k_user', '18_k_user',
       '6FFF_k_user', '10FFF_k_user', '6_Daily_Output', '10_Daily_Output',
       '15_Daily_Output', '18_Daily_Output', '6FFF_Daily_Output',
       '10FFF_Daily_Output', '6__TPR_20,10', '10__TPR_20,10', '15__TPR_20,10',
       '18__TPR_20,10', '6FFF__TPR_20,10', '10FFF__TPR_20,10', 'version',
       'corr.', 'Alg.', 'Rev.',
       'Metric:', 'Case:', 'Depth:', 'Energy:', 'Energy_FFF :', 'Gantry:',
       'Mode:', 'Unidentified_column :', 'predictedValue']]

In [22]:
combinedAll.to_excel("result clean data.xlsx",index = False,na_rep="NA")
combinedAll.to_csv("result clean data.csv",index=False,sep = "\t",na_rep="NA")